In [1]:
!git clone https://github.com/ltt14/Cats-and-Dogs-Classification-using-SVM.git

Cloning into 'Cats-and-Dogs-Classification-using-SVM'...
fatal: could not read Username for 'https://github.com': No such device or address


In [2]:
import numpy as np
#import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report

About the Data


In [3]:
!apt-get install -y libcublas-11-0 libcublas-dev-11-0


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Package libcublas-11-0 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

Package libcublas-dev-11-0 is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source

E: Package 'libcublas-11-0' has no installation candidate
E: Package 'libcublas-dev-11-0' has no installation candidate


In [ ]:
!wget -q https://raw.githubusercontent.com/rapidsai/rapidsai-csp-utils/main/colab/install_rapids.py


In [ ]:
!pip install cuml-cu12==23.12 --extra-index-url=https://pypi.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.2/955.2 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 MB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.9/82.9 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 MB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 88.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.5/509.5 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.7/198.7 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.

In [ ]:
import os
import urllib.request
import tarfile
import pickle
import cv2

# URL of CIFAR-10 dataset
url = "https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz"

# Directory to save downloaded dataset
save_directory = "./cifar-10"

# Create directory if it doesn't exist
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Download and extract CIFAR-10 dataset
filename = url.split("/")[-1]
urllib.request.urlretrieve(url, os.path.join(save_directory, filename))
tar = tarfile.open(os.path.join(save_directory, filename), "r:gz")
tar.extractall(save_directory)
tar.close()

# Define labels for cats and dogs
cat_label = 3
dog_label = 5

# Directory to save cat and dog images
cat_directory = os.path.join(save_directory, "cats")
dog_directory = os.path.join(save_directory, "dogs")

# Create directories if they don't exist
for directory in [cat_directory, dog_directory]:
    if not os.path.exists(directory):
        os.makedirs(directory)

# Extract and save cat and dog images
for filename in os.listdir(os.path.join(save_directory, "cifar-10-batches-py")):
    if "data_batch" in filename:
        with open(os.path.join(save_directory, "cifar-10-batches-py", filename), 'rb') as file:
            data_dict = pickle.load(file, encoding='bytes')
            for i, label in enumerate(data_dict[b'labels']):
                image = data_dict[b'data'][i]
                image = image.reshape((3, 32, 32)).transpose(1, 2, 0)
                if label == cat_label:
                    cv2.imwrite(os.path.join(cat_directory, f"cat_{i}.png"), image)
                elif label == dog_label:
                    cv2.imwrite(os.path.join(dog_directory, f"dog_{i}.png"), image)

print("Images downloaded and saved successfully.")

In [ ]:
# Define the path to the CIFAR-10 dataset folder
dataset_folder = "./cifar-10"

# Function to load and preprocess the images
def load_data():
    images = []
    labels = []
    for category in ['dogs', 'cats']:
        category_folder = os.path.join(dataset_folder, category)
        for filename in os.listdir(category_folder):
            img = cv2.imread(os.path.join(category_folder, filename))
            img = cv2.resize(img, (100, 100))  # Resize image to 100x100
            img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale
            images.append(img.flatten())  # Flatten image into 1D array
            labels.append(category)
    return np.array(images), np.array(labels)

# Load data
X, y = load_data()

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
!pip uninstall cuml-cu12


Found existing installation: cuml-cu12 23.6.0
Uninstalling cuml-cu12-23.6.0:
  Would remove:
    /usr/local/lib/python3.10/dist-packages/cuml/*
    /usr/local/lib/python3.10/dist-packages/cuml_cu12-23.6.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/cuml_cu12.libs/libgomp-a34b3233.so.1.0.0
Proceed (Y/n)? y
  Successfully uninstalled cuml-cu12-23.6.0


In [ ]:

# Install the version of cuml that supports CUDA 12.2
!pip install cuml-cu12==23.6.0 --extra-index-url=https://pypi.nvidia.com


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com


In [ ]:
# Install RAPIDS (if not already installed)

import cuml
from cuml.svm import SVC
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Generate synthetic data
X, y = make_classification(n_samples=1000, n_features=20, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Convert data to GPU arrays
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')
y_train = np.asarray(y_train).astype('int32')
y_test = np.asarray(y_test).astype('int32')

# Initialize cuML SVM classifier
svm_gpu = SVC(kernel="linear", probability=True, random_state=42)

# Train the model
svm_gpu.fit(X_train, y_train)

# Evaluate the model
accuracy = svm_gpu.score(X_test, y_test)
print(f"Accuracy on test data: {accuracy * 100:.2f}%")


/usr/local/lib/python3.10/dist-packages/cupy/_environment.py:540: UserWarning: 
--------------------------------------------------------------------------------

  CuPy may not function correctly because multiple CuPy packages are installed
  in your environment:

    cupy-cuda11x, cupy-cuda12x

  Follow these steps to resolve this issue:

    1. For all packages listed above, run the following command to remove all
       existing CuPy installations:

         $ pip uninstall <package_name>

      If you previously installed CuPy via conda, also run the following:

         $ conda uninstall cupy

    2. Install the appropriate CuPy package.
       Refer to the Installation Guide for detailed instructions.

         https://docs.cupy.dev/en/stable/install.html

--------------------------------------------------------------------------------

  warnings.warn(f'''


ImportError: libcublas.so.11: cannot open shared object file: No such file or directory

In [ ]:
# Initialize SVM classifier
from sklearn.svm import SVC
svm = SVC(kernel='linear', random_state=42)

# Train the model
svm.fit(X_train, y_train)

In [ ]:
# Make predictions
y_pred = svm.predict(X_test)

# Calculate accuracy
from sklearn.metrics import accuracy_score, classification_report
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
print("Classification Report:")
print(classification_report(y_test, y_pred))